# RAG Pipeline for Q&A over a Text File

This notebook implements a clean Retrieval-Augmented Generation (RAG) pipeline.

1.  **Install** required libraries.
2.  **Load** an `OPENAI_API_KEY` (if available).
3.  **Load** a source `.txt` file.
4.  **Chunk, Embed, & Store** the text in a Chroma vector database.
5.  **Build** a LangChain RAG chain to answer questions.
6.  **Run** an interactive chat loop.


In [ ]:
## 1) Install dependencies
import sys
print(sys.version)

# Core libs
!pip -q install langchain langchain-community chromadb sentence-transformers

# For optional local LLM fallback
!pip -q install transformers accelerate

# For OpenAI
!pip -q install langchain-openai




In [ ]:
## 1) Install dependencies
import sys
print(sys.version)

# Core libs
!pip -q install langchain langchain-community chromadb sentence-transformers
!pip -q install langchain-openai
!pip -q install pyttsx3 speechrecognition
!pip -q install langdetect  # <--- NEW LIBRARY

In [1]:
## 2) Load API Key
from pathlib import Path
from dotenv import load_dotenv

# Look for .env in the current working directory (same folder as the notebook)
env_path = Path(".") / ".env"
load_dotenv(dotenv_path=env_path)


True

In [2]:
## 3) Set Constants & Check Key
import os
from pathlib import Path

# Path where Chroma (vector DB) will be persisted
CHROMA_DIR = "./chroma"
COLLECTION = "uploaded_text"

# --- Optional: OpenAI ---
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "").strip()
USE_OPENAI = bool(OPENAI_API_KEY)

if USE_OPENAI:
    print("✅ Using OpenAI for generation.")
else:
    print("ℹ️ OPENAI_API_KEY not set — will use local Transformers fallback.")

Path(CHROMA_DIR).mkdir(parents=True, exist_ok=True)
print("CHROMA_DIR =", Path(CHROMA_DIR).resolve())
print("COLLECTION  =", COLLECTION)


✅ Using OpenAI for generation.
CHROMA_DIR = E:\Lessons_By_Week\Project_Rag\Final_Codes\chroma
COLLECTION  = uploaded_text


In [3]:
## 4) Load Text Document
from pathlib import Path

# Use a relative path. Ensure "RAG_TEXT.txt" is in the same folder as this notebook.
uploaded_path = Path("./RAG_TEXT.txt") 

if not uploaded_path.exists():
    print(f"⚠️ Error: File not found at {uploaded_path.resolve()}")
    print("Please make sure 'RAG_TEXT.txt' is inside the project folder.")
else:
    text = uploaded_path.read_text(encoding="utf-8", errors="ignore")
    print(f"✅ Loaded {len(text):,} characters.")

✅ Loaded 22,028 characters.


In [4]:
## 5) Define LLM (Generator)

generator = None

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    generator = llm
    print("Using ChatOpenAI: gpt-4o-mini")
else:
    # Local Transformers text2text generation via HF pipeline
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
    print("Loading local model: google/flan-t5-base...")
    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    class HFText2TextLLM:
        def __call__(self, prompt_text: str) -> str:
            out = hf_pipe(prompt_text, max_new_tokens=256, truncation=True)
            return out[0]["generated_text"]
    
    generator = HFText2TextLLM()
    print("Using local Transformers: flan-t5-base")


Using ChatOpenAI: gpt-4o-mini


In [5]:
## 6) Chunk, Embed, and Store in Vector DB (Standard Mode)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
import shutil

# --- 1. Chunk the Text ---
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""],
)
docs = [Document(page_content=c, metadata={"source": str(uploaded_path.name)}) 
        for c in splitter.split_text(text)]
print(f"Chunks created: {len(docs)}")

# --- 2. Initialize Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True},
)

# --- 3. Create Vector Store (Chroma) ---
# Force a clean reload of the DB to ensure data is fresh
if Path(CHROMA_DIR).exists():
    shutil.rmtree(CHROMA_DIR)

vs = Chroma(
    collection_name=COLLECTION,
    persist_directory=CHROMA_DIR,
    embedding_function=embeddings,
)
vs.add_documents(docs)
print("✅ Stored in Chroma at:", Path(CHROMA_DIR).resolve())

# --- 4. Standard Retriever (No Threshold) ---
# This will ALWAYS return the closest text, even if it's not very relevant.
retriever = vs.as_retriever(
    search_type="similarity",  # Changed from 'similarity_score_threshold'
    search_kwargs={"k": 5}     # Removed 'score_threshold'
)

print("\n✅ Created Standard Retriever (No safety filter).")

Chunks created: 86


C:\Users\Sadiq\AppData\Local\Temp\ipykernel_17832\3239698443.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


C:\Users\Sadiq\AppData\Local\Temp\ipykernel_17832\3239698443.py:30: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


✅ Stored in Chroma at: E:\Lessons_By_Week\Project_Rag\Final_Codes\chroma

✅ Created Standard Retriever (No safety filter).


In [14]:
## 7) Build Conversational RAG Chain (With Memory)

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory



# --- 1. Contextualize Question ---
# This prompt helps the LLM understand follow-up questions (e.g., "What about him?")
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history.Tell user politly the the history does not exit, "
    
)

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# --- 2. Answer Question ---
# This is the prompt that actually answers the user
qa_system_prompt = (
   "Always start by introducing yourself ''"
   "And summarizing the knowledge base. "
    "You are a here to help discussed the tools *Give summary of the tools in the text*. **Be concise.** "
    "Do not call it text, say it is a knowlegde base"
     "Give the initial summary of what you here for "
    "Do not tell anyone you were trainned by Open AI"
    "You are a helpful assistant. Answer the question only from the provided context. "
    "You may engage in friendly conversation, but never fabricate facts outside the context. "
    "Analyze the text and always give good sumaries and action point available"
    "You answer should be concise"
    "Limit the use of tokens and be very concise"
    "You are a helpful assistant named Chiity. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "IMPORTANT: Answer in the same language that the user asks the question. "
    "If they ask in Spanish, answer in Spanish. If in English, answer in English."


    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# --- 3. Build the Chain ---
if USE_OPENAI:
    # Create a retriever that can handle history
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    
    # Create the document combining chain
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    
    # Combine them
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # --- 4. Memory Management ---
    store = {}

    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]

    conversational_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    print("✅ Conversational RAG (OpenAI + Memory) is ready.")

else:
    # Local Fallback (Simplified memory for local testing)
    # Note: Local models often struggle with the complex history-rewriting step
    print("ℹ️ Memory is disabled for local fallback to ensure stability.")
    conversational_chain = None 
    # We will handle the fallback logic in the ask function

✅ Conversational RAG (OpenAI + Memory) is ready.


In [ ]:
## 8 & 9) Multilingual Chat (Fixed Audio for Jupyter)

import time
import speech_recognition as sr
from gtts import gTTS
import io
from IPython.display import Audio, display

# --- 1. Interactive Language Selector ---
print("🌐 LANGUAGE SELECTION / SELECCIÓN DE IDIOMA")
print("1. English (en)")
print("2. Spanish (es)")
print("3. French  (fr)")
print("4. German  (de)")
print("5. Hindi   (hi)")

choice = input("\n👉 Choose your language (e.g., 'en', 'es'): ").strip().lower()

# Map inputs to (Google Speech Code, gTTS Code, Display Name)
lang_map = {
    'en': {'name': 'English', 'listen': 'en-US', 'speak': 'en'},
    'es': {'name': 'Spanish', 'listen': 'es-ES', 'speak': 'es'},
    'fr': {'name': 'French',  'code': 'fr-FR',   'speak': 'fr'},
    'de': {'name': 'German',  'code': 'de-DE',   'speak': 'de'},
    'hi': {'name': 'Hindi',   'code': 'hi-IN',   'speak': 'hi'}
}

# Default to English if invalid choice
selected = lang_map.get(choice, lang_map['en'])
LANG_NAME = selected['name']
LISTEN_CODE = selected.get('listen', 'en-US') # What Google listens for
SPEAK_CODE = selected.get('speak', 'en')      # What gTTS speaks in

print(f"✅ System set to: {LANG_NAME}")

# --- 2. Audio Functions (Jupyter Compatible) ---
def speak_text(text, lang):
    """Converts text to speech using Google TTS (Works perfectly in Notebooks)"""
    if not text: return
    try:
        # Create audio in memory
        tts = gTTS(text=text, lang=lang, slow=False)
        audio_data = io.BytesIO()
        tts.write_to_fp(audio_data)
        audio_data.seek(0)
        
        # Auto-play the audio
        display(Audio(audio_data.read(), autoplay=True))
    except Exception as e:
        print(f"🔇 Audio Error: {e}")

def listen_instantly():
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        print(f"🔴 LISTENING ({LANG_NAME})...")
        r.adjust_for_ambient_noise(source, duration=0.5)
        try:
            # Listen
            audio = r.listen(source, timeout=5, phrase_time_limit=15)
            print("⏳ Processing...")
            # Transcribe in the selected language
            return r.recognize_google(audio, language=LISTEN_CODE)
        except sr.UnknownValueError:
            print("🤷 Didn't catch that.")
            return None
        except Exception as e:
            print(f"⚠️ Error: {e}")
            return None

# --- 3. Main Loop ---
if 'conversational_chain' not in globals():
    print("⚠️ Brain not loaded. Run previous cells first.")
else:
    # Update System Prompt to enforce language (Optional but recommended)
    # You can manually update Cell 7 if you want strict language enforcement
    
    session_id = f"session_{choice}_v2"
    print("="*50)
    print(f"🎙️ READY! Press [ENTER] to speak in {LANG_NAME}")
    print("="*50)

    try:
        while True:
            user_input = input(f"\n👉 [ENTER]=Voice ({choice}) | [Type]=Text: ").strip()
            if user_input.lower() in ["exit", "quit", "salir"]: 
                print("👋 Goodbye!")
                break
            
            # 1. Input
            query = user_input if user_input else listen_instantly()
            if not query: continue

            print(f"🗣️ You: {query}")
            
            # 2. Brain
            response = conversational_chain.invoke(
                {"input": query},
                config={"configurable": {"session_id": session_id}}
            )
            answer = response["answer"]
            
            # 3. Speak (In the correct language)
            print(f"\n🤖 {answer}\n")
            speak_text(answer, SPEAK_CODE)
            
            # Small delay to prevent mic from picking up the bot's voice immediately
            time.sleep(1)

    except KeyboardInterrupt:
        print("\n🛑 Stopped.")

🌐 LANGUAGE SELECTION / SELECCIÓN DE IDIOMA
1. English (en)
2. Spanish (es)
3. French  (fr)
4. German  (de)
5. Hindi   (hi)
✅ System set to: English
🎙️ READY! Press [ENTER] to speak in English
🔴 LISTENING (English)...
⏳ Processing...
🤷 Didn't catch that.
🔴 LISTENING (English)...
⏳ Processing...
🗣️ You: hello

🤖 Hello! I'm Chiity, here to help you with insights and discussions based on the knowledge base. How can I assist you today?



🔴 LISTENING (English)...
⏳ Processing...
🗣️ You: what is the know

🤖 The knowledge base discusses relationships, focusing on emotional intelligence, communication, and personal growth. It emphasizes the importance of patience, willingness to learn, and healing in relationships. Key points include:

1. Listening and emotional intelligence are crucial in relationships.
2. Both partners should be willing to wait and learn from each other.
3. Healing and personal growth are part of the relationship journey.
4. There is no perfect partner; relationships help in the healing process.

How can I assist you further?



🔴 LISTENING (English)...
⏳ Processing...
🗣️ You: give me a summary

🤖 The knowledge base highlights important lessons about love and relationships, particularly for those in their 20s. It emphasizes:

1. The impact of media on perceptions of love.
2. The importance of emotional intelligence and communication.
3. The need for patience and willingness to learn in relationships.
4. The significance of healing and personal growth.

These insights aim to improve relationships and reduce emotional pain. How else can I help you?



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 1. Define the "LLM-as-Judge" Prompt ---
EVAL_PROMPT_TEMPLATE = """
You are an expert evaluator for a Question-Answering system.
Your goal is to assess whether the 'Generated Answer' correctly and faithfully answers the 'User Question' based *only* on the 'Ground Truth Answer'.

RULES:
- If the 'Generated Answer' is consistent with, and supported by, the 'Ground Truth Answer', respond with **CORRECT**.
- If the 'Generated Answer' contradicts, fabricates information, or misses the main point of the 'Ground Truth Answer', respond with **INCORRECT**.
- If the 'Generated Answer' says "I don't know" and the 'Ground Truth' says the info is missing, this is **CORRECT**.

--- TASK ---
User Question: {question}
Ground Truth Answer: {ground_truth}
Generated Answer: {generated_answer}

Assessment (CORRECT or INCORRECT):"""

eval_prompt = ChatPromptTemplate.from_template(EVAL_PROMPT_TEMPLATE)

# Create the Evaluation Chain
# We reuse 'llm' (ChatOpenAI) if available, otherwise the local 'generator'
judge_llm = llm if USE_OPENAI else generator

if USE_OPENAI:
    evaluation_chain = eval_prompt | judge_llm | StrOutputParser()
else:
    # Fallback wrapper for local model
    def eval_local(inputs):
        txt = eval_prompt.format(**inputs)
        return judge_llm(txt)
    evaluation_chain = eval_local

# --- 2. Define a RELEVANT Test Set (Matches your Uploaded Text) ---
# I have updated this to match the "Content Hub / VFX" content seen in your logs.
evaluation_test_set = [
    {
        "question": "What is ConformPulls?",
        "ground_truth_answer": "ConformPulls is a tool that automates the request for original camera files to reduce delays and manages framing for renders."
    },
    {
        "question": "Is SSTV mentioned on the document",
        "ground_truth_answer": "It handles uploads of camera files, scans directories for errors, organizes media formats, and allows progress tracking from anywhere."
    },
    {
        "question": "Who is Chiity?",
        "ground_truth_answer": "Chiity is the name of the helpful assistant here to discuss the tools in the knowledge base."
    },
    {
        "question": "How do I bake a cake?",
        "ground_truth_answer": "The document does not contain information about cooking or baking cakes."
    }
]

print("✅ Judge Persona & Test Data Loaded.")

In [ ]:
import time

def run_evaluation():
    print(f"📉 Starting Evaluation on {len(evaluation_test_set)} test cases...\n")
    
    score_card = []
    
    for i, item in enumerate(evaluation_test_set):
        q = item["question"]
        gt = item["ground_truth_answer"]
        
        print(f"--- Test Case {i+1}: {q} ---")
        
        # 1. Get answer from your RAG Pipeline
        # We use the 'rag_chain' created in Cell 7 (stateless) or invoke conversational_chain
        try:
            # Using the conversational chain with a fresh session ID for isolation
            response = conversational_chain.invoke(
                {"input": q}, 
                config={"configurable": {"session_id": f"eval_session_{i}"}}
            )
            generated_text = response["answer"]
        except Exception as e:
            generated_text = f"Error generating response: {e}"

        print(f"🤖 Bot Answer: {generated_text[:100]}...") # Print first 100 chars
        
        # 2. Ask the Judge to score it
        eval_input = {
            "question": q,
            "ground_truth": gt,
            "generated_answer": generated_text
        }
        
        if USE_OPENAI:
            grade = evaluation_chain.invoke(eval_input).strip()
        else:
            grade = evaluation_chain(eval_input).strip()
            
        print(f"👨‍⚖️ Judge: {grade}\n")
        score_card.append(grade)
        time.sleep(1) # Avoid rate limits

    # Calculate Final Score
    correct_count = sum(1 for g in score_card if "CORRECT" in g.upper())
    accuracy = (correct_count / len(score_card)) * 100
    
    print("="*30)
    print(f"🏁 Final Accuracy: {accuracy:.1f}% ({correct_count}/{len(score_card)})")
    print("="*30)

# Run it!
run_evaluation()

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ## 1) Install dependencies
import sys
print(sys.version)

# In a .py script, you would typically run these from your terminal first:
# !pip -q install langchain langchain-community chromadb sentence-transformers
# !pip -q install transformers accelerate
# !pip -q install langchain-openai


# ## 2) Load API Key
from pathlib import Path
from dotenv import load_dotenv

# *** UPDATE THIS PATH to your .env file ***
env_path = Path("/Volumes/Untitled/Lessons_By_Week/Project_Rag/Final_Codes/ATT81022.env")
load_dotenv(dotenv_path=env_path)


# ## 3) Set Constants & Check Key
import os
from pathlib import Path

# Path where Chroma (vector DB) will be persisted
CHROMA_DIR = "./chroma"
COLLECTION = "uploaded_text"

# --- Optional: OpenAI ---
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "").strip()
USE_OPENAI = bool(OPENAI_API_KEY)

if USE_OPENAI:
    print("✅ Using OpenAI for generation.")
else:
    print("ℹ️ OPENAI_API_KEY not set — will use local Transformers fallback.")

Path(CHROMA_DIR).mkdir(parents=True, exist_ok=True)
print("CHROMA_DIR =", Path(CHROMA_DIR).resolve())
print("COLLECTION  =", COLLECTION)


# ## 4) Load Text Document

# *** UPDATE THIS PATH to your .txt file ***
uploaded_path = "/Volumes/Untitled/Youtube_QA_Rag_System/Working_Pipelines/text/RAG_TEXT.txt"
from pathlib import Path

p = Path(uploaded_path).expanduser()
assert p.exists(), f"File not found: {p}"

text = p.read_text(encoding="utf-8", errors="ignore")
print(f"Loaded {len(text):,} characters from:", p.resolve())


# ## 5) Define LLM (Generator)

generator = None

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    generator = llm
    print("Using ChatOpenAI: gpt-4o-mini")
else:
    # Local Transformers text2text generation via HF pipeline
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
    print("Loading local model: google/flan-t5-base...")
    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    class HFText2TextLLM:
        def __call__(self, prompt_text: str) -> str:
            out = hf_pipe(prompt_text, max_new_tokens=256, truncation=True)
            return out[0]["generated_text"]
    
    generator = HFText2TextLLM()
    print("Using local Transformers: flan-t5-base")


# ## 6) Chunk, Embed, and Store in Vector DB

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1) Chunk the text
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""],
)
docs = [Document(page_content=c, metadata={"source": str(p.name)}) 
        for c in splitter.split_text(text)]
print(f"Chunks created: {len(docs)}")

# 2) Embedding function
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True},
)

# 3) Create (or re-open) the Chroma collection
vs = Chroma(
    collection_name=COLLECTION,
    persist_directory=CHROMA_DIR,
    embedding_function=embeddings,
)

# 4) Add docs
vs.add_documents(docs)
print("✅ Stored in Chroma at:", Path(CHROMA_DIR).resolve())

# 5) Create the retriever
retriever = vs.as_retriever(search_kwargs={"k": 5})
print("\n✅ Created 'retriever' variable.")


# ## 7) Build RAG Chain

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    out = []
    for i, d in enumerate(docs):
        src = d.metadata.get("source", "")
        out.append(f"[{i}] {d.page_content}\n(source: {src})")
    return "\n\n".join(out)

SYSTEM_PROMPT = (
    "You are a helpful assistant. Answer the question **only** from the provided context.however you can give general friend conversations "
    "If the answer isn't present, say: 'I don't see that in the file.'"
      "store the last 2 questions for retreval '"
     "give full context to make the user understand '"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}"),
    ("human", "Question: {question}\n\nContext:\n{context}\n\nAnswer succinctly:"),
])

# Build chain
if USE_OPENAI:
    chain = (
        RunnableParallel({
            "context": (retriever | format_docs),
            "question": RunnablePassthrough(),
            "system": (lambda _: SYSTEM_PROMPT),
        })
        | prompt
        | generator
        | StrOutputParser()
    )
    print("✅ RAG chain (OpenAI) is ready.")
else:
    # Emulate the same behavior for the local model in a function
    def answer_local(question: str) -> str:
        ctx = format_docs(retriever.get_relevant_documents(question))
        full_prompt = (
            f"{SYSTEM_PROMPT}\n\n"
            f"Question: {question}\n\n"
            f"Context:\n{ctx}\n\n"
            f"Answer succinctly:"
        )
        return generator(full_prompt)

    chain = answer_local
    print("✅ RAG function (Local Transformers) is ready.")


# ## 8) Ask Questions (Interactive)

# Define the 'ask' function
def ask(question: str):
    if not question.strip():
        return "Please enter a non-empty question."
    if callable(chain) and not hasattr(chain, "invoke"):
        # Local HF function path
        return chain(question)
    # OpenAI path via LangChain
    return chain.invoke(question)


# ---
# ## 9) Evaluation Module
# ---

# 1. Define your Test Set
# *** UPDATE THIS TEST SET with questions and answers relevant to your document ***

evaluation_test_set = [
    {
        "question": "What is the file about?",
        "ground_truth_answer": "The file is a comprehensive guide for travelers exploring Europe, focusing on its diverse cultural and natural landscapes, history, art, and the blend of tradition and modernity."
    },
    {
        "question": "What is the Netherlands known for?",
        "ground_truth_answer": "The Netherlands is known for its canals, tulip fields, and cycling culture. Key attractions include Amsterdam’s Rijksmuseum and Anne Frank House."
    },
    {
        "question": "Where is the Netherlands located?",
        "ground_truth_answer": "The Netherlands is located in Western Europe."
    },
    {
        "question": "What is the capital of Spain?",
        "ground_truth_answer": "The document mentions Spain's cultural heritage, architecture, and historical sites, but it does not explicitly state the capital city."
    }
]

print(f"Loaded {len(evaluation_test_set)} evaluation questions.")


# 2. Define the Evaluation Function (LLM-as-Judge)

EVAL_PROMPT_TEMPLATE = """
You are an expert evaluator for a Question-Answering system. 
Your goal is to assess whether the 'Generated Answer' correctly and faithfully answers the 'User Question' based *only* on the 'Ground Truth Answer'.

RULES:
- If the 'Generated Answer' is consistent with, and supported by, the 'Ground Truth Answer', respond with **CORRECT**.
- If the 'Generated Answer' contradicts, fabricates information, or misses the main point of the 'Ground Truth Answer', respond with **INCORRECT**.
- If the 'Generated Answer' is something like 'I don't see that in the file' and the 'Ground Truth Answer' also indicates the information is missing, this is **CORRECT**.

--- EXAMPLES ---
User Question: What is the capital of France?
Ground Truth Answer: Paris is the capital of France.
Generated Answer: The capital of France is Paris.
Assessment: CORRECT

User Question: What is the capital of France?
Ground Truth Answer: Paris is the capital of France.
Generated Answer: I don't see that in the file.
Assessment: INCORRECT

User Question: What is the capital of Mars?
Ground Truth Answer: The document does not mention the capital of Mars.
Generated Answer: I don't see that in the file.
Assessment: CORRECT
--- END EXAMPLES ---

Provide only the final assessment ('CORRECT' or 'INCORRECT').

--- TASK ---
User Question: {question}
Ground Truth Answer: {ground_truth}
Generated Answer: {generated_answer}

Assessment:"""

eval_prompt = ChatPromptTemplate.from_template(EVAL_PROMPT_TEMPLATE)

# Note: We re-use the 'generator' (LLM) from Cell 5 as our judge
if USE_OPENAI:
    evaluation_chain = (
        eval_prompt
        | generator 
        | StrOutputParser()
    )
    print("✅ LLM-as-Judge (OpenAI) is ready.")
else:
    # Local models require the full prompt to be built manually
    def eval_local(inputs: dict) -> str:
        full_prompt = eval_prompt.format(**inputs)
        return generator(full_prompt)
    
    evaluation_chain = eval_local
    print("✅ LLM-as-Judge (Local Transformers) is ready.")


def evaluate_pipeline():
    print("Running evaluation...")
    print("="*30)
    
    correct